In [ ]:
import numpy as np
import twint
import matplotlib.pyplot as plt
import pandas as pd
import re
from random import randrange

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import nest_asyncio
nest_asyncio.apply()

import nltk
nltk.download(['stopwords'])
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
import itertools, collections
from nltk import bigrams

In [ ]:
def read_tweets(search_text, year, tweet_filter, file_name, limit_for_month = 1000):
    
    print('=======================================')
    print(' '.join(['Searching for the term', search_text, 'in', year]))
    
    config = twint.Config()
    config.Search = search_text
    config.Lang = 'en'
    config.Limit = limit_for_month
    config.Store_csv = True
    config.Show_hashtags = True
    config.Output = file_name + year + '.csv'
    if (int(year) >= 2013 and tweet_filter):
        config.Min_likes = 1
    if (int(year) >= 2019 and tweet_filter):
        config.Min_replies = 1
    config.Geo = '39,-98,2400km'
    config.Popular_tweets = True
    config.Count = True
    config.Hide_output = True
    
    months = [f"{x:02d}" for x in range(1, 13)]

    since_list = ['-'.join([year, month, '1']) for month in months]
    until_list = ['-'.join([year, month, '28']) for month in months]

    since_until_list = list(zip(since_list, until_list))
    
    for (since, until) in since_until_list:
        config.Since = since
        config.Until = until
        twint.run.Search(config)

In [ ]:
def read_yearly_tweets(years, tweet_filter, search_terms, file_name):
    for year in years:
        for search_term in search_terms: 
            read_tweets(search_term, year, tweet_filter, file_name, 1000)

In [ ]:
years = [f"{x:02d}" for x in range(2010, 2021)]
search_terms = ['Cannabis', 'Ganja', 'Marijuana']
file_name = 'cannabis/Cannabis'

#years = [f"{x:02d}" for x in range(2013, 2020)]
#search_terms = ['smoking cigarettes', 'tobacco']
#file_name = 'tobacco/Tobacco'

In [ ]:
read_yearly_tweets(years, True, search_terms, file_name)

In [ ]:
def read_users_data(years, file_name):
    for year in years:
        df_full = pd.read_csv(file_name + year + '.csv')
        df_u = df_full[['user_id']]
        df_u.drop_duplicates()
        user_ids = df_u.sample(1000)['user_id'].tolist()
        
        for i, user_id in enumerate(user_ids):
            if i % 20 == 0: print('========= (' + year + ') Step number: ' + str(i) + ' =========')
            config = twint.Config()
            config.Store_csv = True
            config.Output = 'Users' + file_name + year + '.csv'
            config.Hide_output = True
            config.User_id = user_id
            twint.run.Lookup(config)

In [ ]:
years = ['2010', '2014', '2016', '2020']
file_name = 'Cannabis'

In [ ]:
read_users_data(years, file_name)